<a href="https://colab.research.google.com/github/annemariet/tutorials/blob/master/Correction_of_03_ML_and_dataviz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML4Dataviz / Dataviz4ML, let's practice!

This practical is organized in two parts. **The first is "ML4Dataviz"**: we are going to explore the MNIST dataset using high-dimensional projections. The goal is that you get a feeling of how complex datasets can be visualized through classical linear and non-linear projections.

- You are expected to write mostly the `dataviz` part of the code (ie `matplotlib`, `seaborn`, etc)
- You are expected to play with the parameters of the projection functions and explain how they impact performance & display.

**The second part is "Dataviz4ML"**: We will play with the Fashion-MNIST dataset. A simple feed-forward neural network is implemented. Knowing `keras` is not a prerequisite. The goal in this section is to work you through some examples of how visualization will help you understand & debug your models.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn import random_projection, decomposition, manifold
import numpy as np
from time import time

import matplotlib.pyplot as plt
from matplotlib import offsetbox

# 1. Visualizing MNIST

We use the `scikit-learn` library to download the data and matplotlib for preliminary basic visualizations.

In [ ]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True)
X = X / 255.
y = y.astype(np.int)

For simplicity we will only work on the first 1000 samples. Not that the usual split uses the first 60k samples for training and the rest for test.

In [ ]:
n_vis_samples = 1000
w, h = 28, 28
X_vis = X[:n_vis_samples]
y_vis = y[:n_vis_samples]

It's a good idea to have a look at the raw data to begin with. In the specific case of images we can use a the matplotlib function `imshow` and choose the colormap we prefer. 

Check the difference between `gray` and `gray_r` colormaps.



In [ ]:
ax = plt.subplot(111)
ax.imshow(X_vis[0,:].reshape(w, h), cmap=plt.cm.gray_r)
ax.set_title(y_vis[0])
ax.set_axis_off();

We're going to plot images often, so it might help to have a function for this. Define a function plot_image using the following prototype:

In [ ]:
def plot_image(img, reshape_size=None, cmap="binary", ax=None, title=None):
  if ax is None:
    ax = plt.subplot(111)
  if reshape_size is not None:
    img = img.reshape(*reshape_size)
  ax.imshow(img, cmap=cmap)
  if title is not None:
    ax.set_title(title)
  ax.set_axis_off();
  return ax

In [ ]:
plot_image(X_vis[0, :], reshape_size=(w, h), cmap=plt.cm.gray_r, title=y_vis[0])

Now you can use this function to plot a bunch of images from the dataset in a grid and have a feeling of what MNIST digits look like. you can play around with the parameters such as colormaps, grid size, etc..

You can use the [`plt.subplots`](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.subplots.html) from matplotlib to draw several axes on the same figure.

In [ ]:
n_img_per_row = 20
fig, axes = plt.subplots(n_img_per_row // 4, n_img_per_row, figsize=(25,8))
for i in range(n_img_per_row // 4):
  for j in range(n_img_per_row):
    im_index = n_img_per_row * i + j
    plot_image(X_vis[im_index], 
               reshape_size=(w, h),
               #cmap=plt.cm.gray_r,
               ax=axes[i, j],
               title=y_vis[im_index])
plt.suptitle("MNIST")

## High-dimensional projections to 2D

In this exercise we are going to use classical methods to visualize MNIST data in 2D.

The following sample code is adapted from the [scikit-learn documentation](https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html#sphx-glr-auto-examples-manifold-plot-lle-digits-py).

What is the original dimension of MNIST features?

In [ ]:
X_vis.shape[1]

We define a helper function to plot the digits in `y` at positions in `X2d`. The goal of the exercise is to compute different 2D positions for the same original image vectors and get an intuition about how useful they are.

In [ ]:
def plot_digits_embedding(X2d, y, title=None, remove_ticks=True):
  """
  Plot a 2D points at positions `X2d` using text labels from `y`.
  The data is automatically centered and rescaled to [0,1].
  Ticks are removed by default since the axes usually have no meaning (except for PCA).
  """
  x_min, x_max = np.min(X2d, 0), np.max(X2d, 0)
  X = (X2d - x_min) / (x_max - x_min)

  plt.figure(figsize=(20,10))
  ax = plt.subplot(111)
  for i in range(X.shape[0]):
    plt.text(X[i, 0], X[i, 1], str(y[i]),
                color=plt.cm.tab10(y[i]),
                fontdict={'weight': 'bold', 'size': 9})

  if remove_ticks:
    plt.xticks([]), plt.yticks([])
  if title is not None:
    plt.title(title)

We start with a random projection. Even though projecting randomly in 2 dimensions is a rather bad idea, when the original dimension is huge, a random projection to a smaller, still high, dimension might give a nice speedup.

In [ ]:
print("Computing random projection")
t0 = time()
rp = random_projection.SparseRandomProjection(n_components=2, random_state=42)
X_projected = rp.fit_transform(X_vis)
plot_digits_embedding(X_projected, y_vis, "Random Projection of the digits (time %.2fs)" %
               (time() - t0))


### PCA

PCA is the go-to method for dimensionality reduction (not only for visualisation). Play around with it to get a feel of how it works.

You can use scikit-learn's PCA. What is it using under the hood?

In [ ]:
print("Computing PCA projection")
t0 = time()
pca = decomposition.PCA(n_components=2, svd_solver="arpack")
X_pca = pca.fit_transform(X_vis)
plot_digits_embedding(X_pca, y_vis, 
               "Principal Components projection of the digits (time %.2fs)" %
               (time() - t0),
               remove_ticks=False)

*Note that zeros are mostly on the X-axis (around 0.8, 06) and ones are mostly on the second (0, 0.8)*

Compare the different `svd_solver` efficiency. `%timeit` is a notebook magic that allows you to compute the time taken by a line of code. Which method is best if you use the full dataset, vs only a few? (a few: arpack or randomized, full: randomized)

In [ ]:
%timeit decomposition.PCA(n_components=2, svd_solver="full").fit_transform(X)

In [ ]:
%timeit decomposition.PCA(n_components=2, svd_solver="arpack").fit_transform(X)

In [ ]:
%timeit decomposition.PCA(n_components=2, svd_solver="randomized").fit_transform(X)

Since PCA is a linear decomposition, each component is itself an image. Let's have a look at these "eigenimages". They are stored in the `components_` attribute of the sklearn `PCA` object.

In [ ]:
ax = plt.subplot(121)
plot_image(pca.components_[0,:].reshape(w, h), ax=ax, title="PCA first axis")
ax=plt.subplot(122)
plot_image(pca.components_[1,:].reshape(w, h), ax=ax, title="PCA second axis")

Do you think the projection is good? Can you quantify how good it is? How can you explain this? 

In [ ]:
pca.explained_variance_ratio_

_PCA_ is limited to linear transformations of the data. There is little chance that you could find only 2 directions to span images which live in a high-dimensional space. 
Can non-linear methods give us better results here?

### Multidimensional Scaling (MDS)

In [ ]:
print("Computing MDS embedding")
clf = manifold.MDS(n_components=2, n_init=1, max_iter=100)
t0 = time()
X_mds = clf.fit_transform(X_vis)
print("Done. Stress: %f" % clf.stress_)
plot_digits_embedding(X_mds[:n_vis_samples], y_vis, 
               "MDS embedding of the digits (time %.2fs)" %
               (time() - t0))

*Pas très convaincant, bien qu'on voie quelques chiffres se regrouper.*

### t-SNE

Play with the various parameters and observe how time/cluster change. [Scikit-learn documentation](https://scikit-learn.org/stable/modules/manifold.html#t-sne) is a good starting point if you want to know more.

- perplexity (`perplexity`)
- early exaggeration factor (`early_exaggeration`)
- learning rate (`learning_rate`)
- maximum number of iterations (`n_iter`)
- angle (`angle`, not used in the exact method)

Which parameters have more impact?

In [ ]:
tsne = manifold.TSNE(n_components=2, init='random', random_state=42, perplexity=30)
t0 = time()

X_tsne = tsne.fit_transform(X[:5000]) 

plot_digits_embedding(X_tsne[:n_vis_samples], y_vis, 
               "t-SNE embedding of the digits (time %.2fs)" %
               (time() - t0))


### UMAP

UMAP is not yet integrated into scikit-learn but is available as a standalone library that follows the sklearn API. You can have a look at [their doc](https://umap-learn.readthedocs.io/en/latest/parameters.html) to get a better understanding of the parameters.

In [ ]:
#!pip install umap-learn # run this if you run into an "ModuleNotFoundError: No module named 'umap'" error below

In [ ]:
import umap
t0 = time()
um = umap.UMAP(n_neighbors=20, min_dist=0.1, metric="euclidean")
#um.fit_transform(X_vis)  # faster, but fitting with more samples below give a cleaner output
X_umap = um.fit_transform(X[:5000])
t1 = time()
fig_umap = plot_digits_embedding(X_umap[:n_vis_samples], y_vis, 
               "UMAP embedding of the digits (time %.2fs)" %
               (t1 - t0))

In this example, UMAP yields a much cleaner plot, and is faster.

If time permits, you can try combining the methods above: for instance, use PCA to reduce the dimensionality to accelerate either MDS or tSNE. What do you observe?

In [ ]:
#TODO_OPTIONAL
t0 = time()
# Use PCA to get only 50 dimensions, then UMAP
pca = decomposition.PCA(n_components=10, svd_solver="auto")
tsne = manifold.TSNE(n_components=2, init='random', random_state=42, perplexity=30)

X_pca = pca.fit_transform(X[:5000])
X_tsne = tsne.fit_transform(X_pca) 
t1 = time()
fig_tsne_pca = plot_digits_embedding(X_tsne[:n_vis_samples], y_vis, 
               "PCA-TSNE embedding of the digits (time %.2fs)" %
               (t1 - t0))


*The result is about the same as with TSNE, but twice faster. Note that 784 is not that big and the speed-up may vary depending on the methods you use, especially as it also depends on the number of points.*

## The same plots with Bokeh

We are going to improve the visualization of `X_projected`, `X_pca`, `X_mds` and `X_tsne` by using `bokeh`.


If the Bokeh interactive plots don't show as expected, you might have to revert to a previous version of Bokeh (run `!pip install bokeh==1.3.4`) and restart your runtime.

In [ ]:
import bokeh
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tools import HoverTool
import matplotlib as mpl
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

You are provided with two helper functions to transform images to bokeh-compatible formats. Replace the call to `circle` with `image` and `image_rgba` to change the glyphs.

In [ ]:

def im_colorize_rgba(arr, label):
  color = mpl.cm.tab10(label)
  if arr.ndim == 1:
    out_img = ((1 - arr[:, np.newaxis]) * np.array(color)).reshape(28,28,4) * 255
  elif arr.ndim == 2:
    out_img = (arr[:, :, np.newaxis] * np.array(color)) * 255
  else:
    raise ValueError("Image format not handled (yet).")
  out_img = np.flipud(out_img)
  return np.squeeze(out_img.astype(np.uint8)).view(np.uint32)

def im_reshape_bw(arr):
  out_img = (1 - arr.reshape(28,28))*255
  out_img = np.flipud(out_img)
  return out_img
  
def plot_interactive_embedding(X, y, X_img, title=None, dscale=3):
  """
  X: 2D projection of the input vectors (floats)
  y: label/category indexes (integers)
  X_img: original input vectors (images as vectors of dimension 784)
  title (optional): plot title
  dscale (default=3): control image glyphs, should be adjusted depending on the scale of X
  """
  bokeh_data = ColumnDataSource(
          data=dict(
              x1 = X[:,0],
              x2 = X[:,1],
              colors = ["#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 
                        255*mpl.cm.tab10(y)],
              images = [im_reshape_bw(xi) for xi in X_img],
              images_rgba = [im_colorize_rgba(xi, yi) for xi, yi in zip(X_img, y)],
              labels = [str(yi) for yi in y]
          )
      )
  hover_tsne = HoverTool(tooltips='@labels')
  tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']
  plot_tsne = figure(plot_width=600, plot_height=600, tools=tools_tsne, title=title)

  #plot_tsne.circle("x1", "x2", size=4, fill_color="colors", line_width=0, source=bokeh_data)
  plot_tsne.image_rgba(image="images_rgba", x="x1", y="x2", dw=dscale, dh=dscale, source=bokeh_data)
  #plot_tsne.image(image="images", x="x1", y="x2", dw=2, dh=2, source=bokeh_data)

  return plot_tsne

In [ ]:
fig_tsne = plot_interactive_embedding(X_tsne[:n_vis_samples], y_vis, X_vis, 
               "t-SNE embedding of the digits")

In [ ]:
show(fig_tsne)

In [ ]:
fig_umap = plot_interactive_embedding(X_umap[:n_vis_samples], y_vis, X_vis, 
               "UMAP embedding of the digits", dscale=0.3)
show(fig_umap)

# 2. Visualizing and debugging neural networks

In the next section, we're going to reproduce what you're learnt before in your machine learning courses with a focus on the visualizing part of it. You should come out with handy functions that you can reuse later for your own experiments.

We will use Fashion-MNIST because that's more fun to visualize.

Build an image classifier & visualize layers

In [ ]:
# adapted from https://github.com/ageron/handson-ml2/blob/master/10_neural_nets_with_keras.ipynb
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
assert tf.__version__ >= "2.0"

from tensorflow import keras

In [ ]:
tf.__version__

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
X_train_full.dtype

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
plot_image(X_train[0], reshape_size=None, cmap="binary", ax=None, title=class_names[y_train[0]]);

In [ ]:
n_rows = 5
n_cols = 20
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        ax = plt.subplot(n_rows, n_cols, index + 1)
        plot_image(X_train[index], ax=ax, title=class_names[y_train[index]]);
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [ ]:
n_vis_samples = 1000
X_vis = X_test[:n_vis_samples]
y_vis = y_test[:n_vis_samples]
X_in = X_vis.reshape(n_vis_samples, 28*28)

In [ ]:
um = umap.UMAP(n_neighbors=20, min_dist=0.1, metric="euclidean")
X_2D = um.fit_transform(X_in)
fig_2D_orig = plot_interactive_embedding(X_2D, y_vis, X_vis, 
               "UMAP embedding of fashion MNIST from image space", dscale=0.5)
show(fig_2D_orig)

## A simple fully connected neural network

In [ ]:
model_fc = keras.models.Sequential()
model_fc.add(keras.layers.Flatten(input_shape=[28, 28]))
model_fc.add(keras.layers.Dense(50, activation="relu", name="fc1"))
model_fc.add(keras.layers.Dense(100, activation="relu", name="fc2"))
model_fc.add(keras.layers.Dense(10, activation="softmax", name="cls_1"))

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
keras.utils.plot_model(model_fc, "my_mnist_model.png", show_shapes=True)

In [ ]:
model_fc.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
# before fitting
layer_to_plot = ["fc1", "fc2"]
features = {}
X_in = X_vis.reshape(n_vis_samples, 28*28)
for l in model_fc.layers:
  X_out = l(X_in)
  X_in = X_out
  if l.name in layer_to_plot:
    features[l.name] = X_out

X_2D = um.fit_transform(features["fc2"])
fig_2D_fc2_rand = plot_interactive_embedding(X_2D, y_vis, X_vis,
               "UMAP embedding of fashion MNIST from feature space fc2, without training", dscale=0.5)
show(fig_2D_fc2_rand)

*This is not exactly like a random projection, but basically it doesn't look like the structure of the data has changed much.*

In [ ]:
# training takes about 1 minute
history = model_fc.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

### Looking at the learned weights directly

The first layer's weights is a $784\times h$ matrix, so we can observe a few lines of this matrix as images. Can you see patterns emerge?

*It looks like most of the work is done by the first layer -- this may need the network needs longer training time, or better tuning.*

In [ ]:
layer_weights = model_fc.get_weights()
len(layer_weights), layer_weights[0].shape, layer_weights[1].shape, layer_weights[2].shape

In [ ]:
fig, axes = plt.subplots(4, 4)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = layer_weights[0].min(), layer_weights[0].max()

for coef, ax in zip(layer_weights[0].T, axes.ravel()):
    ax.matshow(coef.reshape(28, 28), cmap=plt.cm.gray, vmin=.5 * vmin,
               vmax=.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())

*Even though the images are very noisy, this doesn't look like pure noise, there are indeed faint patterns encoded in the first layer's parameters.*

### Looking at the latent representations

Another way to look at the intermediate layers is through their effect as feature transforms.

Plot a view of the points as transformed at different levels in the network. What can you observe?

_We would expect that categories get easier to separate as we go deeper in the network_.

In [ ]:
# after fitting
layer_to_plot = ["fc1", "fc2"]
features = {}
X_in = X_vis.reshape(n_vis_samples, 28*28)
for l in model_fc.layers:
  X_out = l(X_in)
  X_in = X_out
  if l.name in layer_to_plot:
    features[l.name] = X_out


X_2D = um.fit_transform(features["fc1"])
fig_2D_fc1 = plot_interactive_embedding(X_2D, y_vis, X_vis, 
               "UMAP embedding of fashion MNIST from feature space fc1, _after_ training", dscale=0.5)

X_2D = um.fit_transform(features["fc2"])
fig_2D_fc2 = plot_interactive_embedding(X_2D, y_vis, X_vis, 
               "UMAP embedding of fashion MNIST from feature space fc2, _after_ training", dscale=0.5)


In [ ]:
show(fig_2D_fc1)

In [ ]:
show(fig_2D_fc2)

### Debugging Learning

Plot the learning curve from the history output of the `fit` call. What do you observe?

In [ ]:
import pandas as pd
import seaborn as sns

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.gca().set_ylim(0, 1)
plt.show()
sns.despine()

_Answer: the learning curve is plateauing, it might be a good idea to lower the learning rate (ReduceLROnPlateau, or use some schedule)._

### What do predictions look like?

In this part, we're going to look at the output of the classification model itself.

In [ ]:
# Look at a few random predictions
n_test = 10
X_new = X_test[:n_test]
y_proba = model_fc.predict(X_new)
y_proba.round(2)
y_pred = np.argmax(model_fc.predict(X_new), axis=-1)

In [ ]:
# This could be turned into a function
plt.figure(figsize=(20, 3))
for index, image in enumerate(X_new):
    ax = plt.subplot(1, n_test, index + 1)
    title="\n".join((class_names[y_test[index]], class_names[y_pred[index]]))
    plot_image(image, ax=ax, title=title);
plt.show()

#### The distribution of predictions

You can use `seaborn`, `bokeh`, or raw `matplotlib` to view these distributions. It might be useful to organise the predictions with dataframes.

1. Which classes are predicted the more, the less?
2. What is the distribution of the predicted probabilities for each class? Do you see any trend? Does it raise questions?

In [ ]:
# compute test predictions
y_proba_test = model_fc.predict(X_test)
y_pred = y_proba_test.argmax(axis=-1)

In [ ]:
ax = plt.subplot(111)
ax.hist([y_pred.astype(int), y_test], rwidth=0.7, bins=np.arange(11), label=['predicted', 'true'])
ax.set_xticks(np.array(range(10))+0.5)
plt.xticks(rotation=75)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax.set_xticklabels(class_names);

*Answer 1: Pullover is under-predicted, while Coat is overpredicted.*

In [ ]:
# looking at all the probabilities output vectors
# we expect something like 90% close to 0, 10% close to one, maybe more in-between if there is uncertainty in predictions
df = pd.DataFrame(data=y_proba_test, columns=class_names)
df = df.stack()
df.index.names = ["id", "class_name"]
df.name = "predicted"
df_tidy = df.reset_index(level=1)

In [ ]:
df2 = pd.DataFrame(data=y_test, columns=["true"])
df_tidy = df.reset_index(level=1).join(df2)
df_tidy.sort_values(by="class_name", inplace=True)

In [ ]:
df_tidy.head()

In [ ]:
df_tidy.groupby("class_name").describe()

In [ ]:
# adapted from https://seaborn.pydata.org/examples/kde_ridgeplot.html
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(df_tidy, row="class_name", hue="class_name", aspect=10, height=0.8, palette=pal, sharey=False)

# Draw the densities in a few steps
#g.map(sns.distplot, "predicted", kde=False, hist=True, rug=False, bins=100)
g.map(sns.kdeplot, "predicted", clip_on=False, shade=True, alpha=1, lw=3) #, bw=.05)
g.map(plt.axhline, y=0, lw=5, clip_on=False)
def label(x, color, label):
    ax = plt.gca()
    ax.text(-0.1, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

g.map(label, "predicted")

# Set the subplots to overlap
g.fig.subplots_adjust(hspace=-.02)
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)

*Answer 2 (trends): Unless you use a different scale on the y-axis, you can't see much differnce between classes, only that the divide is so clear between high and low predictions.
Looking with different scales helps: the classification is much easier for some classes (as could be expected by looking at the clusters on the feature projections).*

*Next we'll restrict to the max proba for each sample to zoom on the proba of actual predictions.*

In [ ]:
y_proba_max = y_proba_test[np.arange(len(y_pred)), y_pred]
df = pd.DataFrame(data=np.array([y_proba_max, y_pred, y_test]).T, columns=["proba", "predicted", "true"])
df[["predicted", "true"]] = df[["predicted", "true"]].astype(int)
df["class_name"] = df["predicted"].apply(lambda x: class_names[x])
df.sort_values(by="class_name", inplace=True)
df.head()

In [ ]:
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(df, row="class_name", hue="class_name", aspect=10, height=0.8, palette=pal, sharey=False)

# Draw the densities in a few steps
#g.map(sns.distplot, "predicted", kde=False, hist=True, rug=False, bins=100)
g.map(sns.kdeplot, "proba", clip_on=False, shade=True, alpha=1, lw=3) #, bw=.05)
g.map(plt.axhline, y=0, lw=5, clip_on=False)
def label(x, color, label):
    ax = plt.gca()
    ax.text(-0.1, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

g.map(label, "predicted")

# Set the subplots to overlap
g.fig.subplots_adjust(hspace=-.02)
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)

*There are clearly 2 kinds of classes: the easy ones (Ankle boot, Bag, Sandal, Sneaker, Trouser) and the hard ones, especially Shirt and Pullover.*

*We expect to see this in other ways below*.

#### Which examples are the hardest to classify?

Plot hard example and check whether you'd agree with the model (that they are hard).

In [ ]:
# Find worst mistakes
y_proba_test = model_fc.predict(X_test)
mistakes = np.squeeze(np.argwhere(y_pred != y_test))
worst_mistakes = np.argsort(-y_proba_test[mistakes,:].max(axis=1))[:n_test]

In [ ]:
y_pred[mistakes[:10]], y_test[mistakes[:10]]

In [ ]:
y_pred[worst_mistakes], y_test[worst_mistakes]

In [ ]:
plt.figure(figsize=(20, 3))
for i, index in enumerate(mistakes[worst_mistakes]):
    ax = plt.subplot(1, n_test, i + 1)
    title="\n".join((f"true: {class_names[y_test[index]]}", f"pred: {class_names[y_pred[index]]}"))
    plot_image(X_test[index], ax=ax, title=title);
plt.show()

#### Which classes are the most difficult to classify? The most confused?

This kind of question is best answered by looking at the confusion matrix. If you don't know how to do that, you can look at `sklearn.metrics.confusion_matrix` and `sns.heatmap` for help.

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred, normalize="true")
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
plt.figure(figsize = (10,8))
sns.heatmap(df_cm, annot=True, cmap="viridis");

*We could almost tell it based on the distribution of probabilities: Pullover and Shirt/T-Shirt are the hardest, then Dress/Coat, the others are pretty easy.*

#Visualizing CNN models

In [ ]:
from PIL import Image
from functools import partial
import os


In [ ]:
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train_c = (X_train - X_mean) / X_std
X_valid_c = (X_valid - X_mean) / X_std
X_test_c = (X_test - X_mean) / X_std

X_train_c = X_train_c[..., np.newaxis]
X_valid_c = X_valid_c[..., np.newaxis]
X_test_c = X_test_c[..., np.newaxis]

In [ ]:

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

feature_extractor = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
])
classifier = keras.models.Sequential([
    keras.layers.Dense(units=128, activation='selu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='selu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

model = keras.models.Sequential([feature_extractor, classifier])

*Let's first look at the embedding produced by _random_ convolution filters, yay!*.

In [ ]:
X_2D = um.fit_transform(feature_extractor(X_vis[:,:,:, np.newaxis]))
fig_2D_cnn = plot_interactive_embedding(X_2D, y_vis, X_vis, 
               "UMAP embedding of fashion MNIST from feature space fc2, _before_ training", dscale=0.5)
show(fig_2D_cnn)

*Plot the kernel weights (first layer only, but you get the idea). Before training, it is not very interesting, it's randomly initialized.*

In [ ]:
layer_weights = feature_extractor.get_weights()
fig, axes = plt.subplots(8, 8)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = layer_weights[0].min(), layer_weights[0].max()

for coef, ax in zip(layer_weights[0].T, axes.ravel()):
    k1 = (coef.reshape(7, 7) - vmin) / (vmax - vmin)
    im = Image.fromarray(k1 * 255)
    ax.imshow(im.resize((28,28)))
    ax.set_xticks(())
    ax.set_yticks(())


In [ ]:
# This is much longer than the feed-forward network so we train only for 10 epochs
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train_c[:10000], y_train[:10000], epochs=10, validation_data=(X_valid_c, y_valid))

In [ ]:
score = model.evaluate(X_test_c, y_test)
X_new = X_test_c[:10] # pretend we have new images
y_pred = model.predict(X_new)


You can redo the same visualization as with the feed-forward model. Instead of looking at weights as images, you can plot the CNN's kernel weights as tiny images.

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.gca().set_ylim(0, 1)
plt.show()
sns.despine()

*There is a risk of overfitting here... this is an example, hyper-parameters have not been tuned.*

In [ ]:
X_2D = um.fit_transform(feature_extractor(X_vis[:,:,:, np.newaxis]))
fig_2D_cnn = plot_interactive_embedding(X_2D, y_vis, X_vis, 
               "UMAP embedding of fashion MNIST from CNN feature space, _after_ training", dscale=0.5)
show(fig_2D_cnn)

In [ ]:
show(fig_2D_fc2)

In [ ]:
layer_weights = feature_extractor.get_weights()
fig = plt.figure(figsize=(15, 10))
axes = fig.subplots(8, 8)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = layer_weights[0].min(), layer_weights[0].max()

for coef, ax in zip(layer_weights[0].T, axes.ravel()):
    k1 = (coef.reshape(7, 7) - vmin) / (vmax - vmin)
    im = Image.fromarray(k1 * 255)
    ax.imshow(im.resize((28,28)))
    ax.set_xticks(())
    ax.set_yticks(())


*I have seen cleaner filters than this, but still, given that the network is not finely tuned, it's not bad: for instance we can observe some directional filters (vertical, diagnoal)*.

This is the most direct primitive way to watch at what the CNN does. Kernels are not always easy to interpret, especially when they combine to produce the results. Which is why people look at how they affect some test images. One example methods which is history already is DeepDream.

If time permits, you can go through the [DeepDream tutorial](https://github.com/tensorflow/tensorflow/blob/r0.10/tensorflow/examples/tutorials/deepdream/deepdream.ipynb) and adapt it here to visualise the layers of your network.